In [16]:
# Import libraries
import panel as pn
import pandas as pd
import geopandas as gpd
import plotly.express as px
import json
import plotly.graph_objects as go
from pathlib import Path

    

In [17]:
import pandas as pd

# Load the CSV with the second row as headers
df = pd.read_csv('/workspaces/Uganda_education/primary_education_dataset/primary_indicators.csv', header=1)

# Rename columns with year suffixes
columns = df.columns
new_columns = [
    'District',
    'Gross Enrollment Ratio_2014', 'Pupil Teacher Ratio_2014', 'Primary Completion Ratio_2014',
    'Gross Enrollment Ratio_2015', 'Pupil Teacher Ratio_2015', 'Primary Completion Ratio_2015',
    'Gross Enrollment Ratio_2016', 'Pupil Teacher Ratio_2016', 'Primary Completion Ratio_2016',
    'Gross Enrollment Ratio_2017', 'Pupil Teacher Ratio_2017', 'Primary Completion Ratio_2017'
]
df.columns = new_columns

# Convert to long format
df_long = pd.melt(
    df,
    id_vars=['District'],
    value_vars=[
        'Gross Enrollment Ratio_2014', 'Pupil Teacher Ratio_2014', 'Primary Completion Ratio_2014',
        'Gross Enrollment Ratio_2015', 'Pupil Teacher Ratio_2015', 'Primary Completion Ratio_2015',
        'Gross Enrollment Ratio_2016', 'Pupil Teacher Ratio_2016', 'Primary Completion Ratio_2016',
        'Gross Enrollment Ratio_2017', 'Pupil Teacher Ratio_2017', 'Primary Completion Ratio_2017'
    ],
    var_name='Metric_Year',
    value_name='Value'
)

# Split Metric_Year into Metric and Year
df_long[['Metric', 'Year']] = df_long['Metric_Year'].str.extract(r'(.*)_(\d{4})')
df_long = df_long.drop(columns=['Metric_Year'])

# Clean and standardize
df_long['Metric'] = df_long['Metric'].str.strip()
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')
df_long['District'] = df_long['District'].str.upper()

# Filter to match GeoJSON districts
with open('uganda_districts.json') as f:
    uganda_geojson = json.load(f)
geo_districts = [f['properties']['District'] for f in uganda_geojson['features']]
df_long = df_long[df_long['District'].isin(geo_districts)]

# Save the full dataset
df_long.to_csv('education_data_long.csv', index=False)

# Verify
print(df_long.head())
print(df_long['District'].unique())

       District  Value                  Metric  Year
0        BUIKWE   85.0  Gross Enrollment Ratio  2014
1  BUKOMANSIMBI  131.0  Gross Enrollment Ratio  2014
2     BUTAMBALA  125.0  Gross Enrollment Ratio  2014
3        BUVUMA  198.0  Gross Enrollment Ratio  2014
4         GOMBA  111.0  Gross Enrollment Ratio  2014
['BUIKWE' 'BUKOMANSIMBI' 'BUTAMBALA' 'BUVUMA' 'GOMBA' 'KALANGALA'
 'KALUNGU' 'KAMPALA' 'KAYUNGA' 'KIBOGA' 'KYOTERA' 'LWENGO' 'LYANTONDE'
 'MASAKA' 'MITYANA' 'MPIGI' 'MUBENDE' 'MUKONO' 'NAKASEKE' 'NAKASONGOLA'
 'RAKAI' 'SSEMBABULE' 'WAKISO' 'BUDAKA' 'BUDUDA' 'BUGIRI' 'BUKEDEA'
 'BUKWO' 'BULAMBULI' 'BUSIA' 'BUTALEJA' 'BUYENDE' 'IGANGA' 'JINJA'
 'KABERAMAIDO' 'KALIRO' 'KAMULI' 'KAPCHORWA' 'KATAKWI' 'KIBUKU' 'KUMI'
 'KWEEN' 'LUUKA' 'MANAFWA' 'MAYUGE' 'MBALE' 'NAMAYINGO' 'NAMISINDWA'
 'NAMUTUMBA' 'NGORA' 'PALLISA' 'SERERE' 'SIRONKO' 'SOROTI' 'TORORO' 'ABIM'
 'ADJUMANI' 'AGAGO' 'ALEBTONG' 'AMOLATAR' 'AMUDAT' 'AMURIA' 'AMURU' 'APAC'
 'ARUA' 'DOKOLO' 'GULU' 'KAABONG' 'KITGUM' 'KOBO

In [18]:
df_long.to_csv('education_data_long.csv', index=False)

# Verify the saved data
df_verified = pd.read_csv('education_data_long.csv')
print(df_verified.head())
print(df_verified['District'].unique())
print(f"Total rows: {len(df_verified)}")
print(f"Unique districts: {len(df_verified['District'].unique())}")
print(f"Unique metrics: {df_verified['Metric'].unique()}")
print(f"Unique years: {df_verified['Year'].unique()}")

       District  Value                  Metric  Year
0        BUIKWE   85.0  Gross Enrollment Ratio  2014
1  BUKOMANSIMBI  131.0  Gross Enrollment Ratio  2014
2     BUTAMBALA  125.0  Gross Enrollment Ratio  2014
3        BUVUMA  198.0  Gross Enrollment Ratio  2014
4         GOMBA  111.0  Gross Enrollment Ratio  2014
['BUIKWE' 'BUKOMANSIMBI' 'BUTAMBALA' 'BUVUMA' 'GOMBA' 'KALANGALA'
 'KALUNGU' 'KAMPALA' 'KAYUNGA' 'KIBOGA' 'KYOTERA' 'LWENGO' 'LYANTONDE'
 'MASAKA' 'MITYANA' 'MPIGI' 'MUBENDE' 'MUKONO' 'NAKASEKE' 'NAKASONGOLA'
 'RAKAI' 'SSEMBABULE' 'WAKISO' 'BUDAKA' 'BUDUDA' 'BUGIRI' 'BUKEDEA'
 'BUKWO' 'BULAMBULI' 'BUSIA' 'BUTALEJA' 'BUYENDE' 'IGANGA' 'JINJA'
 'KABERAMAIDO' 'KALIRO' 'KAMULI' 'KAPCHORWA' 'KATAKWI' 'KIBUKU' 'KUMI'
 'KWEEN' 'LUUKA' 'MANAFWA' 'MAYUGE' 'MBALE' 'NAMAYINGO' 'NAMISINDWA'
 'NAMUTUMBA' 'NGORA' 'PALLISA' 'SERERE' 'SIRONKO' 'SOROTI' 'TORORO' 'ABIM'
 'ADJUMANI' 'AGAGO' 'ALEBTONG' 'AMOLATAR' 'AMUDAT' 'AMURIA' 'AMURU' 'APAC'
 'ARUA' 'DOKOLO' 'GULU' 'KAABONG' 'KITGUM' 'KOBO

In [19]:
import pandas as pd

df = pd.read_csv('education_data_long.csv')
enrollment_df = df[df['Metric'] == 'Gross Enrollment Ratio'].pivot(index=['District', 'Year'], columns='Metric', values='Value').reset_index()
completion_df = df[df['Metric'] == 'Primary Completion Ratio'].pivot(index=['District', 'Year'], columns='Metric', values='Value').reset_index()
efficiency_df = pd.merge(enrollment_df, completion_df, on=['District', 'Year'], how='inner')
efficiency_df['Efficiency'] = (efficiency_df['Primary Completion Ratio'] / efficiency_df['Gross Enrollment Ratio'] * 100).fillna(0)
print(efficiency_df[['District', 'Year', 'Gross Enrollment Ratio', 'Primary Completion Ratio', 'Efficiency']].sort_values('Efficiency', ascending=False).head(10))  # Top 10
print(efficiency_df[['District', 'Year', 'Gross Enrollment Ratio', 'Primary Completion Ratio', 'Efficiency']].sort_values('Efficiency', ascending=True).head(10))  # Bottom 10

Metric District  Year  Gross Enrollment Ratio  Primary Completion Ratio  \
23       AMUDAT  2017                    21.0                      76.0   
22       AMUDAT  2016                    21.0                      68.0   
381       NAPAK  2015                    50.0                     159.0   
21       AMUDAT  2015                    22.0                      68.0   
148     KAABONG  2014                    49.0                     140.0   
20       AMUDAT  2014                    21.0                      59.0   
253      KOTIDO  2015                    42.0                      94.0   
252      KOTIDO  2014                    38.0                      68.0   
255      KOTIDO  2017                    49.0                      84.0   
254      KOTIDO  2016                    44.0                      72.0   

Metric  Efficiency  
23      361.904762  
22      323.809524  
381     318.000000  
21      309.090909  
148     285.714286  
20      280.952381  
253     223.809524  
252   

In [20]:
import pandas as pd

# Load data
df = pd.read_csv('education_data_long.csv')

# Prepare efficiency data (pre-swap)
enrollment_df = df[df['Metric'] == 'Gross Enrollment Ratio'].pivot(index=['District', 'Year'], columns='Metric', values='Value').reset_index()
completion_df = df[df['Metric'] == 'Primary Completion Ratio'].pivot(index=['District', 'Year'], columns='Metric', values='Value').reset_index()
efficiency_df = pd.merge(enrollment_df, completion_df, on=['District', 'Year'], how='inner')
efficiency_df['Efficiency'] = (efficiency_df['Primary Completion Ratio'] / efficiency_df['Gross Enrollment Ratio'] * 100).fillna(0).clip(upper=100)

# Display Amudat data for verification
print(efficiency_df[efficiency_df['District'] == 'AMUDAT'][['District', 'Year', 'Gross Enrollment Ratio', 'Primary Completion Ratio', 'Efficiency']])

Metric District  Year  Gross Enrollment Ratio  Primary Completion Ratio  \
20       AMUDAT  2014                    21.0                      59.0   
21       AMUDAT  2015                    22.0                      68.0   
22       AMUDAT  2016                    21.0                      68.0   
23       AMUDAT  2017                    21.0                      76.0   

Metric  Efficiency  
20           100.0  
21           100.0  
22           100.0  
23           100.0  
